In [1]:
from __future__ import absolute_import, print_function


import tvm
from tvm import te
import tvm.testing
import numpy as np

from tvm.relay.backend import Executor, Runtime

In [5]:
N, M, L = 1024, 512, 64
A = te.placeholder((N, L), name="A")
B = te.placeholder((M, L), name="B")
k = te.reduce_axis((0, L), name="k")
C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[j, k], axis=k), name="C")
s = te.create_schedule(C.op)
print(tvm.lower(s, [A, B, C], simple_mode=True))

@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {C: Buffer(C_2: Pointer(float32), float32, [1024, 512], []),
             B: Buffer(B_2: Pointer(float32), float32, [512, 64], []),
             A: Buffer(A_2: Pointer(float32), float32, [1024, 64], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C} {
  for (i: int32, 0, 1024) {
    for (j: int32, 0, 512) {
      C_2[((i*512) + j)] = 0f32
      for (k: int32, 0, 64) {
        C_2[((i*512) + j)] = ((float32*)C_2[((i*512) + j)] + ((float32*)A_2[((i*64) + k)]*(float32*)B_2[((j*64) + k)]))
      }
    }
  }
}




In [3]:
factor = 16
x, y = C.op.axis
(z,) = C.op.reduce_axis
yo, yi = s[C].split(y, factor=factor)
s[C].reorder(x, yo, yi, z)
print(tvm.lower(s, [A, B, C], simple_mode=True))

compute(C, body=[reduce(combiner=comm_reducer(result=[(x + y)], lhs=[x], rhs=[y], identity_element=[0f]), source=[(A[i, k]*B[j, k])], init=[], axis=[iter_var(k, range(min=0, ext=64))], where=(bool)1, value_index=0)], axis=[iter_var(i, range(min=0, ext=1024)), iter_var(j, range(min=0, ext=512))], reduce_axis=[iter_var(k, range(min=0, ext=64))], tag=, attrs={})
compute(C, body=[reduce(combiner=comm_reducer(result=[(x + y)], lhs=[x], rhs=[y], identity_element=[0f]), source=[(A[i, k]*B[j, k])], init=[], axis=[iter_var(k, range(min=0, ext=64))], where=(bool)1, value_index=0)], axis=[iter_var(i, range(min=0, ext=1024)), iter_var(j, range(min=0, ext=512))], reduce_axis=[iter_var(k, range(min=0, ext=64))], tag=, attrs={})
@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {C: Buffer(C_2: Pointer(float32), float32, [1024, 512], []),
             B: Buffer(B_2: Pointer(float32), float32,

In [4]:
def intrin_gemv(m, l):
    a = te.placeholder((l,), name="a")
    b = te.placeholder((m, l), name="b")
    k = te.reduce_axis((0, l), name="k")
    c = te.compute((m,), lambda i: te.sum(a[k] * b[i, k], axis=k), name="c")
    Ab = tvm.tir.decl_buffer(a.shape, a.dtype, name="A", offset_factor=1, strides=[1])
    Bb = tvm.tir.decl_buffer(b.shape, b.dtype, name="B", offset_factor=1, strides=[te.var("s1"), 1])
    Cb = tvm.tir.decl_buffer(c.shape, c.dtype, name="C", offset_factor=1, strides=[1])

    def intrin_func(ins, outs):
        ib = tvm.tir.ir_builder.create()
        aa, bb = ins
        cc = outs[0]
        ib.emit(
            tvm.tir.call_extern(
                "int32",
                "gemv_update",
                cc.access_ptr("w"),
                aa.access_ptr("r"),
                bb.access_ptr("r"),
                m,
                l,
                bb.strides[0],
            )
        )
        return ib.get()

    return te.decl_tensor_intrin(c.op, intrin_func, binds={a: Ab, b: Bb, c: Cb})


In [5]:
def gemv_impl():
    cc_code = """
      extern "C" int gemv_update(float *cc, float *aa, float *bb, int m, int l, int stride) {
        for (int i = 0; i < m; ++i) {
            for (int j = 0; j < l; ++j) {
                cc[i] += aa[j] * bb[i * stride + j];
            }
        }
        return 0;
      }
    """
    from tvm.contrib import utils, clang

    temp = utils.tempdir()
    ll_path = temp.relpath("temp.ll")
    # Create LLVM ir from c source code
    ll_code = clang.create_llvm(cc_code, output=ll_path)
    return ll_code

In [6]:
gemv = intrin_gemv(factor, L)
s[C].pragma(x, "import_llvm", gemv_impl())
s[C].tensorize(yi, gemv)
module1 = tvm.lower(s, [A, B, C], simple_mode=True)
print(module1)

compute(C, body=[reduce(combiner=comm_reducer(result=[(x + y)], lhs=[x], rhs=[y], identity_element=[0f]), source=[(A[i, k]*B[j, k])], init=[], axis=[iter_var(k, range(min=0, ext=64))], where=(bool)1, value_index=0)], axis=[iter_var(i, range(min=0, ext=1024)), iter_var(j, range(min=0, ext=512))], reduce_axis=[iter_var(k, range(min=0, ext=64))], tag=, attrs={})
compute(C, body=[reduce(combiner=comm_reducer(result=[(x + y)], lhs=[x], rhs=[y], identity_element=[0f]), source=[(A[i, k]*B[j, k])], init=[], axis=[iter_var(k, range(min=0, ext=64))], where=(bool)1, value_index=0)], axis=[iter_var(i, range(min=0, ext=1024)), iter_var(j, range(min=0, ext=512))], reduce_axis=[iter_var(k, range(min=0, ext=64))], tag=, attrs={})
@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {C: Buffer(C_2: Pointer(float32), float32, [1024, 512], []),
             B: Buffer(B_2: Pointer(float32), float32,

[14:35:58] /home/julien/3la-evaluation/3la-tvm/src/printer/doc.cc:55: Warning: text node: '; ModuleID = '/tmp/tmpp0mv7m6q/input0.cc'
source_filename = "/tmp/tmpp0mv7m6q/input0.cc"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

; Function Attrs: noinline nounwind optnone uwtable
define dso_local i32 @gemv_update(float* %0, float* %1, float* %2, i32 %3, i32 %4, i32 %5) #0 {
  %7 = alloca float*, align 8
  %8 = alloca float*, align 8
  %9 = alloca float*, align 8
  %10 = alloca i32, align 4
  %11 = alloca i32, align 4
  %12 = alloca i32, align 4
  %13 = alloca i32, align 4
  %14 = alloca i32, align 4
  store float* %0, float** %7, align 8
  store float* %1, float** %8, align 8
  store float* %2, float** %9, align 8
  store i32 %3, i32* %10, align 4
  store i32 %4, i32* %11, align 4
  store i32 %5, i32* %12, align 4
  store i32 0, i32* %13, align 4
  br label %15

15:                                       

In [12]:
func = tvm.build(s, [A, B, C], target="c", name="gemv", runtime=Runtime("crt", {"system-lib": True}))
file_path = 'tvm_test.c'
from tvm.micro.model_library_format import export_model_library_format
export_model_library_format(func, "tvm_test.tar")

system-lib


[14:56:28] /home/julien/3la-evaluation/3la-tvm/src/printer/doc.cc:55: Warning: text node: '; ModuleID = '/tmp/tmpp0mv7m6q/input0.cc'
source_filename = "/tmp/tmpp0mv7m6q/input0.cc"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-linux-gnu"

; Function Attrs: noinline nounwind optnone uwtable
define dso_local i32 @gemv_update(float* %0, float* %1, float* %2, i32 %3, i32 %4, i32 %5) #0 {
  %7 = alloca float*, align 8
  %8 = alloca float*, align 8
  %9 = alloca float*, align 8
  %10 = alloca i32, align 4
  %11 = alloca i32, align 4
  %12 = alloca i32, align 4
  %13 = alloca i32, align 4
  %14 = alloca i32, align 4
  store float* %0, float** %7, align 8
  store float* %1, float** %8, align 8
  store float* %2, float** %9, align 8
  store i32 %3, i32* %10, align 4
  store i32 %4, i32* %11, align 4
  store i32 %5, i32* %12, align 4
  store i32 0, i32* %13, align 4
  br label %15

15:                                       

PosixPath('tvm_test.tar')

In [ ]:
from tvm.driver.tvmc.model import TVMCModel
from tvm.driver.tvmc.compiler import compile_model
from tvm.relay.backend import Executor, Runtime

model = TVMCModel(module1, {})
compile_model(tvmc_model=model,
                  target="c -device=arm_cpu",
                  executor=Executor("aot",
                                    {"interface-api": "c",
                                     "unpacked-api": 1}
                                    ),
                  runtime=Runtime("crt"),
                  output_format="mlf",
                  package_path="model.tar")